**#Before you run this file , please download the compressed trained model(facemodel.h5) and depress it in the model folder. Then you can just run all blocks and check the real performance of this system. Please adjust your camera so that the stickers are in the right position(need to be improved). And moving too fast might cause exception.**

In [9]:
import cv2
import numpy as np
import os
import sys

IMAGE_SIZE = 128 # we assign the image as (128,128,3)

def resize_image(image, height = IMAGE_SIZE, width = IMAGE_SIZE):
    top, bottom, left, right = (0,0,0,0)
    
    h, w, _ = image.shape
    
    # find the longer edge if the image is not square
    longest_edge = max(h,w)
    
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass 
    
    BLACK = [0,0,0]
    constant = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)
    return cv2.resize(constant, (height, width))

In [10]:
#CNN model
class Model:
    
    def __init__(self):
        self.model = None
        
    def build_model(self, dataset, nb_classes = 5):
        self.model = Sequential()
        self.model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = dataset.input_shape))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(32, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Conv2D(64, (3, 3), padding = 'same'))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(nb_classes))
        self.model.add(Activation('softmax'))
    def train(self, dataset, batch_size = 128, nb_epoch = 6):
        
        self.model.compile(loss = 'categorical_crossentropy', 
                           optimizer = 'ADAM',
                           metrics = ['accuracy'])
        self.model.fit(dataset.train_images, 
                           dataset.train_labels, 
                           batch_size = batch_size,
                           epochs = nb_epoch, 
                           shuffle = True)

    def evaluate(self, dataset):
        score = self.model.evaluate(dataset.test_images, dataset.test_labels)
        print("%s: %.3f%%" % (self.model.metrics_names[1], score[1] * 100))
        
    def save_model(self, file_path):
        self.model.save(file_path)
    def load_model(self, file_path):
        self.model = load_model(file_path)
    def face_predict(self, image):
        #use resize function defined before
        image = resize_image(image)
        image = image.reshape((1, IMAGE_SIZE, IMAGE_SIZE, 3))
        image = image.astype('float32')
        image /= 255
        result = self.model.predict(image)
        return result.argmax(axis=-1)

In [11]:
# Final Method
def get_facecover(x, y, w, h, img, rgb_image, a):
    eyes_center = ((2 * x + w)//2 , (2 * y)//2) # find the the center of the target's eyes
#     factor = 1.2  # manual control size factor  
    resized_image_h = h  # resized_image_h = h * factor
    resized_image_w = w 

    if resized_image_h > y:
        resized_image_h = y-1

    # Resized image based on resized h and w
    resized_image = cv2.resize(rgb_image,(resized_image_w,resized_image_h))
    mask = cv2.resize(a,(resized_image_w,resized_image_h))
    mask_inv =  cv2.bitwise_not(mask)
    # Seted shifing offset
    dh = int(1.015 * h)
    dw = int(-0.2 * w) 
    
    # Find face
    bg_roi = img[y+dh-resized_image_h:y+dh,(eyes_center[0]-resized_image_w//3)+dw:(eyes_center[0]+resized_image_w//3*2)+dw]
    bg_roi = bg_roi.astype(float)
    
    # Generate Mask
    mask_inv = cv2.merge((mask_inv,mask_inv,mask_inv))
    alpha = mask_inv.astype(float)/255

    # resize alpha for future calculation
    alpha = cv2.resize(alpha,(bg_roi.shape[1],bg_roi.shape[0]))
    bg = cv2.multiply(alpha, bg_roi)
    bg = bg.astype('uint8')


    # Get the area of face mask
    image = cv2.bitwise_and(resized_image,resized_image,mask = mask)
    image = cv2.resize(image,(bg_roi.shape[1],bg_roi.shape[0]))
    # Add face mask to the target area
    add_image = cv2.add(bg,image)

    # Replace the original with the face mask
    img[y+dh-resized_image_h:y+dh,(eyes_center[0]-resized_image_w//3)+dw:(eyes_center[0]+resized_image_w//3*2)+dw] = add_image.copy()
    
    # Return the edited frame
    return img

In [12]:
from PIL import Image
facial_file = ['angry','hate','fear','happy','sad','surprise','neutral']

facial_dict = {}
for i in range(7):
    print(i)
    face_img = cv2.imread(facial_file[i] + '.png',cv2.IMREAD_UNCHANGED)
    r,g,b,a = cv2.split(face_img) 
    rgb_face = cv2.merge((r,g,b))
    cv2.imwrite(facial_file[i]+'_alpha.png',a)
    facial_dict[facial_file[i]] = [rgb_face,a]

0
1
2
3
4
5
6


In [13]:
import cv2
import sys
from keras.models import load_model
# Def emotions labels
emotion_labels = {
    0: 'angry',
    1: 'hate',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'surprise',
    6: 'neutral'
}

# Load emotion classifier
emotion_classifier = load_model('model/simple_CNN.530-0.65.hdf5')
# Load face detection model
model = Model()
model.load_model(file_path = './model/facemodel.h5')    

# Create OpenCv window
cv2.namedWindow('Detecting your face.') 
color = (0, 255, 0)

# Load haarcascade classifier
classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
# Capture the real time image frame
cap = cv2.VideoCapture(0)
while cap.isOpened():
        ok, frame = cap.read() # type(frame) <class 'numpy.ndarray'>
        if not ok:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # gray scale the frame
        frame2 = frame.copy()
        faceRects=classifier.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3,minSize=(32,32))
        
        # If detected target face
        if len(faceRects) > 0:
            
            for faceRect in faceRects: 
                
                x, y, w, h = faceRect                
                image = frame[y - 10: y + h + 10, x - 10: x + w + 10].copy()
                
                
                # Dectect if the image is none
                # if we won't do this step, there will be an error:
                    # error error (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize
                if image is None:  
                    break
                else:
                    if image is not None:
                        gray_face = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        gray_face = cv2.resize(gray_face,(48,48))
                        gray_face = gray_face/255.0
                        gray_face =np.expand_dims(gray_face,0)
                        gray_face =np.expand_dims(gray_face,-1)
                        
                        # Predict emotion from the gray scale image
                        emotion_label_arg = np.argmax(emotion_classifier.predict(gray_face))
                        emotion = emotion_labels[emotion_label_arg]
                        faceID = model.face_predict(image)
                        
                        # Make a face mask based on the emotions
                        img = frame2.copy()
                        img = get_facecover(x, y, w, h, img, facial_dict[emotion][0], facial_dict[emotion][1])
                        frame2 = img.copy()
                        
                        # Show the target's name and emotion
                        if faceID[0] == 0:                                                        
                            cv2.rectangle(frame2, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                            cv2.putText(frame2,'Yan is '+ emotion, 
                                    (x + 30, y + 30),                      # Coordinate
                                    cv2.FONT_HERSHEY_SIMPLEX,              # Font
                                    1,                                     # Word size
                                    (255,0,255),                           # Word color
                                    2)                                     # Word's line width
                            
                        elif faceID[0] == 1:
                            cv2.rectangle(frame2, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                            cv2.putText(frame2,'Yujie is ' + emotion, 
                                    (x + 30, y + 30),                     
                                    cv2.FONT_HERSHEY_SIMPLEX,             
                                    1,                                     
                                    (255,0,255),                        
                                    2)                                   
                        elif faceID[0] == 2:
                            cv2.rectangle(frame2, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                            cv2.putText(frame2,'Liang is ' + emotion, 
                                    (x + 30, y + 30),                      
                                    cv2.FONT_HERSHEY_SIMPLEX,             
                                    1,                                  
                                    (255,0,255),                          
                                    2)                                     
                        elif faceID[0] == 4:
                            cv2.rectangle(frame2, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                            cv2.putText(frame2,'Zuxian is ' + emotion, 
                                    (x + 30, y + 30),                     
                                    cv2.FONT_HERSHEY_SIMPLEX,             
                                    1,                                  
                                    (255,0,255),                           
                                    2)                                     
                        else:
                            cv2.rectangle(frame2, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                            cv2.putText(frame2,'Unknown is '+ emotion, 
                                    (x + 30, y + 30),                    
                                    cv2.FONT_HERSHEY_SIMPLEX,            
                                    1,                                   
                                    (255,0,255),                         
                                    2)    
        # Show the edited frame on the screen
                
        cv2.imshow("Detecting your face.", frame2)
        
        # Press to q to exit
       
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

# Release the capture
cap.release()
cv2.destroyAllWindows()